# Model Evaluation The Best Model 

In [3]:
# 📦 Basic Libraries
import os
import zipfile
import gdown
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

# 🎯 Sklearn - Evaluation
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import KFold

# 🔥 PyTorch + timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import timm

# ⚙️ ONNX Runtime
import onnxruntime as ort

# load the  model

In [5]:
# This for using gpu or cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [6]:
# model 
model = timm.create_model('efficientvit_b2.r288_in1k', pretrained=True)
model.to(device)

print("model sucessfully loaded ✅")

model sucessfully loaded ✅


**Training Dataset**

In [10]:
# data prepprocessing 

transform = transforms.Compose([
    transforms.Resize((288, 288)),      
    transforms.ToTensor(),              
])

In [22]:
model.load_state_dict(torch.load("/kaggle/input/efficientvit/pytorch/v1/1/best_model.pth"))
model.eval()

# Load test set
train_dataset = datasets.ImageFolder(root='/kaggle/working/train_data/X_train_image', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)

# Evaluate
correct, total = 0, 0
with torch.no_grad():
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"\n🧪 Test Accuracy: {correct / total:.4f}")

/tmp/ipykernel_31/3893677420.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/kaggle/input/efficientvit/pytorch/v1/1/best_model.pth"))



🧪 Test Accuracy: 0.9926


**TestData Dataset**

In [23]:
model.load_state_dict(torch.load("/kaggle/input/efficientvit/pytorch/v1/1/best_model.pth"))
model.eval()

# Load test set
test_dataset = datasets.ImageFolder(root='/kaggle/working/test_data/X_test_image', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Evaluate
correct, total = 0, 0
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"\n🧪 Test Accuracy: {correct / total:.4f}")

/tmp/ipykernel_31/266595089.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/kaggle/input/efficientvit/pytorch/v1/1/best_model.pth"))



🧪 Test Accuracy: 0.9572


# classification report 

In [25]:
# Store true & predicted labels
y_true = []
y_pred = []

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.to(device)
        outputs = model(imgs)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

# Show classification report
class_names = test_dataset.classes
print("\n📊 Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=class_names))


📊 Classification Report:

                    precision    recall  f1-score   support

    8-cell Grade A       0.98      0.97      0.97        89
    8-cell Grade B       0.94      0.94      0.94        86
    8-cell Grade C       0.95      0.94      0.95        89
Blastocyst Grade A       0.92      1.00      0.96        97
Blastocyst Grade B       0.99      0.85      0.91        98
Blastocyst Grade C       0.92      0.98      0.95        99
      Error Images       1.00      1.00      1.00       100
    Morula Grade A       0.94      0.97      0.96       100
    Morula Grade B       0.96      0.95      0.95       100
    Morula Grade C       0.98      0.97      0.97       100

          accuracy                           0.96       958
         macro avg       0.96      0.96      0.96       958
      weighted avg       0.96      0.96      0.96       958



### cross validation for train dataset

In [26]:
# Dataset
dataset = datasets.ImageFolder(root='/kaggle/working/train_data/X_train_image', transform=transform)

# KFold setup
kfold = KFold(n_splits=5, shuffle=True)
fold_results = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
    print(f"\n🔁 Fold {fold+1}/5")

    train_data = Subset(dataset, train_idx)
    val_data = Subset(dataset, val_idx)

    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

    # Load model
    # model = timm.create_model("efficientvit_b0", pretrained=False, num_classes=10)
    model.load_state_dict(torch.load("/kaggle/input/efficientvit/pytorch/v1/1/best_model.pth"))  # 🔥 Your trained weights
    model.to(device)

    # Evaluate only (no training needed)
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    acc = correct / total
    print(f"✅ Fold {fold+1} Accuracy: {acc:.4f}")
    fold_results.append(acc)

# Final results
print("\n📊 Cross Validation Accuracy")
print("Mean:", np.mean(fold_results), " | Std:", np.std(fold_results))



🔁 Fold 1/5


/tmp/ipykernel_31/2453849207.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/kaggle/input/efficientvit/pytorch/v1/1/best_model.pth"))

✅ Fold 1 Accuracy: 0.9919

🔁 Fold 2/5
✅ Fold 2 Accuracy: 0.9907

🔁 Fold 3/5
✅ Fold 3 Accuracy: 0.9919

🔁 Fold 4/5
✅ Fold 4 Accuracy: 0.9948

🔁 Fold 5/5
✅ Fold 5 Accuracy: 0.9936

📊 Cross Validation Accuracy
Mean: 0.99257634755708  | Std: 0.0014391832525461085


### cross validation for test dataset

In [27]:
# Dataset
dataset = datasets.ImageFolder(root='/kaggle/working/test_data/X_test_image', transform=transform)

# KFold setup
kfold = KFold(n_splits=5, shuffle=True)
fold_results = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
    print(f"\n🔁 Fold {fold+1}/5")

    train_data = Subset(dataset, train_idx)
    val_data = Subset(dataset, val_idx)

    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

    # Load model
    # model = timm.create_model("efficientvit_b0", pretrained=False, num_classes=10)
    model.load_state_dict(torch.load("/kaggle/input/efficientvit/pytorch/v1/1/best_model.pth"))  # 🔥 Your trained weights
    model.to(device)

    # Evaluate only (no training needed)
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    acc = correct / total
    print(f"✅ Fold {fold+1} Accuracy: {acc:.4f}")
    fold_results.append(acc)

# Final results
print("\n📊 Cross Validation Accuracy")
print("Mean:", np.mean(fold_results), " | Std:", np.std(fold_results))



🔁 Fold 1/5


/tmp/ipykernel_31/2273692423.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/kaggle/input/efficientvit/pytorch/v1/1/best_model.pth"))

✅ Fold 1 Accuracy: 0.9479

🔁 Fold 2/5
✅ Fold 2 Accuracy: 0.9635

🔁 Fold 3/5
✅ Fold 3 Accuracy: 0.9427

🔁 Fold 4/5
✅ Fold 4 Accuracy: 0.9581

🔁 Fold 5/5
✅ Fold 5 Accuracy: 0.9738

📊 Cross Validation Accuracy
Mean: 0.9572207678883071  | Std: 0.011077286277875407


# Check with new images

In [ ]:
import onnxruntime as ort
import numpy as np
from PIL import Image
import torchvision.transforms as transforms

# Load ONNX model
session = ort.InferenceSession("/kaggle/working/efficientvit_embryo.onnx")

# Define transform
transform = transforms.Compose([
    transforms.Resize((288, 288)),
    transforms.ToTensor(),
])

# Load image and preprocess
img = Image.open("/kaggle/working/test_data/X_test_image/Morula Grade C/image_233.jpg").convert("RGB")
img_tensor = transform(img).unsqueeze(0).numpy()  # shape: [1, 3, 288, 288]

# Class names
class_names = [
    '8-cell Grade A', '8-cell Grade B', '8-cell Grade C',
    'Blastocyst Grade A', 'Blastocyst Grade B', 'Blastocyst Grade C',
    'Error Images', 'Morula Grade A', 'Morula Grade B', 'Morula Grade C'
]

# Run model
outputs = session.run(None, {"input": img_tensor})
logits = outputs[0]

# Apply softmax
probs = np.exp(logits) / np.sum(np.exp(logits))
pred_class = np.argmax(probs)

# Show result
print("✅ Prediction:", class_names[pred_class])

# Check With New Images

In [40]:
# Load ONNX model
session = ort.InferenceSession("/kaggle/working/efficientvit_embryo_fp16.onnx")

# # Define transform
transform = transforms.Compose([
    transforms.Resize((288, 288)),
    transforms.ToTensor(),
])

# Class names
class_names = [
    '8-cell Grade A', '8-cell Grade B', '8-cell Grade C',
    'Blastocyst Grade A', 'Blastocyst Grade B', 'Blastocyst Grade C',
    'Error Images', 'Morula Grade A', 'Morula Grade B', 'Morula Grade C'
]

In [59]:
 # Load and preprocess image
img = Image.open("/kaggle/input/demoimages/NewImages/istockphoto-1681388313-612x612.jpg").convert("RGB")
img_tensor = transform(img).unsqueeze(0).numpy().astype(np.float16)

# Run inference
outputs = session.run(None, {"input": img_tensor})
logits = outputs[0]

# Apply softmax
probs = np.exp(logits) / np.sum(np.exp(logits))
pred_class = np.argmax(probs)

# Print prediction and confidence
print("✅ Prediction:", class_names[pred_class])
print("📊 Confidence Score:", probs[0][pred_class])

# Print all class probabilities
# print("\n🔍 Class Probabilities:")
# for i, prob in enumerate(probs[0]):
#     print(f"{class_names[i]}: {prob:.4f}")

✅ Prediction: Error Images
📊 Confidence Score: 0.999


In [1]:
import os 